# MongoDB Aggregation Pipeline Demo

![Pipe](pipe.jpg)

In [1]:
%alias mongo "C:\Program Files\MongoDB\Server\3.4\bin\mongo.exe" Demo --eval %s

### Our collection

```JSON
[
	{
	'name': 'Bob',
	'address': {
		'Street': 'Olive Blvd',
		'City': 'Burbank',
		'State': 'CA'
		},
	'ordered': ['lamb', 'veal', 'chicken'],
	'lifetime_value': 450
	},
	{
	'name': 'Sam',
	'address': {
		'Street': 'Magnolia Blvd',
		'City': 'Burbank',
		'State': 'CA'
		},
	'ordered': ['turkey', 'duck', 'chicken'],
	'lifetime_value': 1000
	},
	{
	'name': 'Jess',
	'address': {
		'Street': 'Hollywood Blvd',
		'City': 'Hollywood',
		'State': 'CA'
		},
	'ordered': ['beef', 'lamb', 'chicken'],
	'lifetime_value': 200
	}
	{
	'name': 'Alejandra',
	'address': {
		'Street': 'Segway Dr',
		'City': 'Bloomington',
		'State': 'IN'
		},
	'ordered': ['shrimp', 'beef'],
	'lifetime_value': 2200
	}
]
```

In [2]:
mongo "db.customers.insert([{'name':'Bob','address':{'Street':'Olive Blvd','City':'Burbank','State':'CA'},'ordered':['lamb','veal','chicken'],'lifetime_value':450},{'name':'Sam','address':{'Street':'Magnolia Blvd','City':'Burbank','State':'CA'},'ordered':['turkey','duck','chicken'],'lifetime_value':1000},{'name':'Jess','address':{'Street':'Hollywood Blvd','City':'Hollywood','State':'CA'},'ordered':['beef','lamb','chicken'],'lifetime_value':200},{'name':'Alejandra','address':{'Street':'Segway Dr','City':'Bloomington','State':'IN'},'ordered':['shrimp','beef'],'lifetime_value':2200}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
BulkWriteResult({
	"writeErrors" : [ ],
	"writeConcernErrors" : [ ],
	"nInserted" : 4,
	"nUpserted" : 0,
	"nMatched" : 0,
	"nModified" : 0,
	"nRemoved" : 0,
	"upserted" : [ ]
})


### How many of our customers are from California?

In [3]:
mongo "db.customers.aggregate([{$match:{'address.State': 'CA'}}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : ObjectId("58cad8fd7832184696640ac4"), "name" : "Bob", "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : [ "lamb", "veal", "chicken" ], "lifetime_value" : 450 }
{ "_id" : ObjectId("58cad8fd7832184696640ac5"), "name" : "Sam", "address" : { "Street" : "Magnolia Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : [ "turkey", "duck", "chicken" ], "lifetime_value" : 1000 }
{ "_id" : ObjectId("58cad8fd7832184696640ac6"), "name" : "Jess", "address" : { "Street" : "Hollywood Blvd", "City" : "Hollywood", "State" : "CA" }, "ordered" : [ "beef", "lamb", "chicken" ], "lifetime_value" : 200 }


In [4]:
mongo "db.customers.aggregate([{$match:{'address.State':'CA'}},{$count:'californians'}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "californians" : 3 }


*Mongoose*
```javascript
app.get('/match', function(req, res){
	User.aggregate(
		{$match: {'address.State': 'CA'}},
		{$count: 'californians'}
	, function(err, users){
		if(err){
			console.log(err);
		} else {
			console.log(users);
			return res.json(users);
		}
	});
});
```

### How profitable is Burbank?

#### Group first, then match.

In [5]:
mongo "db.customers.aggregate([{$group:{ _id: '$address.City', total_value: {$sum: '$lifetime_value'}}}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : "Hollywood", "total_value" : 200 }
{ "_id" : "Bloomington", "total_value" : 2200 }
{ "_id" : "Burbank", "total_value" : 1450 }


In [6]:
mongo "db.customers.aggregate([{$group:{ _id: '$address.City', total_value: {$sum: '$lifetime_value'}}}, {$match: {_id: 'Burbank'}}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : "Burbank", "total_value" : 1450 }


*Mongoose*
```javascript
app.get('/group', function(req, res){
	User.aggregate([
		{$group: {
			_id: '$address.City',
			total_value: {$sum: '$lifetime_value'}
		}}]
	, function(err, users){
		if(err){
			console.log(err);
		} else {
			console.log(users);
			return res.json(users);
		}
	})
});
```

#### Match first, then group

In [7]:
mongo "db.customers.aggregate([{$match: { 'address.City': 'Burbank'}}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : ObjectId("58cad8fd7832184696640ac4"), "name" : "Bob", "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : [ "lamb", "veal", "chicken" ], "lifetime_value" : 450 }
{ "_id" : ObjectId("58cad8fd7832184696640ac5"), "name" : "Sam", "address" : { "Street" : "Magnolia Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : [ "turkey", "duck", "chicken" ], "lifetime_value" : 1000 }


In [8]:
mongo "db.customers.aggregate([{$match: { 'address.City': 'Burbank'}}, {$group: {_id: 'address.City', total_value: {$sum: '$lifetime_value'}}}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : "address.City", "total_value" : 1450 }


### Suddenly, we need a separate collection for the orders.

#### Unwind Magic

In [9]:
mongo "db.customers.aggregate([{$unwind: '$ordered'}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : ObjectId("58cad8fd7832184696640ac4"), "name" : "Bob", "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : "lamb", "lifetime_value" : 450 }
{ "_id" : ObjectId("58cad8fd7832184696640ac4"), "name" : "Bob", "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : "veal", "lifetime_value" : 450 }
{ "_id" : ObjectId("58cad8fd7832184696640ac4"), "name" : "Bob", "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : "chicken", "lifetime_value" : 450 }
{ "_id" : ObjectId("58cad8fd7832184696640ac5"), "name" : "Sam", "address" : { "Street" : "Magnolia Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : "turkey", "lifetime_value" : 1000 }
{ "_id" : ObjectId("58cad8fd7832184696640ac5"), "name" : "Sam", "address" : { "Street" : "Magnolia Blvd", "City" : "Burbank", "State" : "CA" }

*Mongoose*
```javascript
app.get('/unwind', function(req, res){
	User.aggregate([
		{$unwind: '$ordered'}
	], function(err, ordered){
		if(err){
			console.log(err);
		} else {
			console.log(ordered);
			return res.json(ordered);
		}
	});
});
```

#### Collections need unique \_ids. Using addFields.

In [10]:
mongo "db.customers.aggregate([{$unwind: '$ordered'}, {$addFields: { _id: {name: '$name', ordered: '$ordered'}}}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : { "name" : "Bob", "ordered" : "lamb" }, "name" : "Bob", "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : "lamb", "lifetime_value" : 450 }
{ "_id" : { "name" : "Bob", "ordered" : "veal" }, "name" : "Bob", "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : "veal", "lifetime_value" : 450 }
{ "_id" : { "name" : "Bob", "ordered" : "chicken" }, "name" : "Bob", "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : "chicken", "lifetime_value" : 450 }
{ "_id" : { "name" : "Sam", "ordered" : "turkey" }, "name" : "Sam", "address" : { "Street" : "Magnolia Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : "turkey", "lifetime_value" : 1000 }
{ "_id" : { "name" : "Sam", "ordered" : "duck" }, "name" : "Sam", "address" : { "Street" : "Magnolia Blvd", "City" : "Burbank", "

#### Creating a new collection within the aggregate pipeline

In [11]:
mongo "db.customers.aggregate([{$unwind: '$ordered'}, {$addFields: { _id: {name: '$name', ordered: '$ordered'}}}, {$out: 'orders'}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2


In [12]:
mongo "db.orders.find()"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : { "name" : "Bob", "ordered" : "lamb" }, "name" : "Bob", "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : "lamb", "lifetime_value" : 450 }
{ "_id" : { "name" : "Bob", "ordered" : "veal" }, "name" : "Bob", "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : "veal", "lifetime_value" : 450 }
{ "_id" : { "name" : "Bob", "ordered" : "chicken" }, "name" : "Bob", "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : "chicken", "lifetime_value" : 450 }
{ "_id" : { "name" : "Sam", "ordered" : "turkey" }, "name" : "Sam", "address" : { "Street" : "Magnolia Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : "turkey", "lifetime_value" : 1000 }
{ "_id" : { "name" : "Sam", "ordered" : "duck" }, "name" : "Sam", "address" : { "Street" : "Magnolia Blvd", "City" : "Burbank", "

#### Redundant and Unnecessary fields! Drop the bass...

In [13]:
mongo "db.orders.drop()"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
true


#### Projecting(similar to second argument in `find()`)

In [14]:
mongo "db.customers.aggregate([{$unwind: '$ordered'}, {$addFields: { _id: {name: '$name', ordered: '$ordered'}}}, {$project: {id: 1, address: 1}}, {$out: 'orders'} ])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2


In [15]:
mongo "db.orders.find()"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : { "name" : "Bob", "ordered" : "lamb" }, "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" } }
{ "_id" : { "name" : "Bob", "ordered" : "veal" }, "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" } }
{ "_id" : { "name" : "Bob", "ordered" : "chicken" }, "address" : { "Street" : "Olive Blvd", "City" : "Burbank", "State" : "CA" } }
{ "_id" : { "name" : "Sam", "ordered" : "turkey" }, "address" : { "Street" : "Magnolia Blvd", "City" : "Burbank", "State" : "CA" } }
{ "_id" : { "name" : "Sam", "ordered" : "duck" }, "address" : { "Street" : "Magnolia Blvd", "City" : "Burbank", "State" : "CA" } }
{ "_id" : { "name" : "Sam", "ordered" : "chicken" }, "address" : { "Street" : "Magnolia Blvd", "City" : "Burbank", "State" : "CA" } }
{ "_id" : { "name" : "Jess", "ordered" : "beef" }, "address" : { "Street" : "Hollywood Blvd", "City" : "Hollyw

*Mongoose*
```javascript
app.get('/out', function(req, res){
	User.aggregate([
		{$unwind: '$ordered'},
		{$addFields: {_id: {name: '$name', ordered: '$ordered'}}},
		{$project: {id: 1, address: 1}},
		{$out: "orders"}
	], function(err, coll){
		if(err){
			console.log(err);
		} else {
			mongoose.connection.db.collection('orders', function(err, collection){
				collection.find({}).toArray(function(err, orders){
					if(err){
						console.log(err);
					} else {
						for(var i = 0; i < orders.length; i++){
							quantity = Math.round(Math.random()*3)+1;
							orders[i].quantity = quantity;
							collection.save(orders[i]);
						}
						return res.json(orders);
					}
				});
			});
		}
	});
});
```

### Suddenly, we recieve quantities(finally)

In [16]:
mongo "var orders=db.orders.find(); orders.forEach(function(order){order.quantity=Math.round(Math.random()*3)+1;db.orders.save(order)})"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2


In [17]:
mongo "db.orders.find().pretty()"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{
	"_id" : {
		"name" : "Bob",
		"ordered" : "lamb"
	},
	"address" : {
		"Street" : "Olive Blvd",
		"City" : "Burbank",
		"State" : "CA"
	},
	"quantity" : 2
}
{
	"_id" : {
		"name" : "Bob",
		"ordered" : "veal"
	},
	"address" : {
		"Street" : "Olive Blvd",
		"City" : "Burbank",
		"State" : "CA"
	},
	"quantity" : 2
}
{
	"_id" : {
		"name" : "Bob",
		"ordered" : "chicken"
	},
	"address" : {
		"Street" : "Olive Blvd",
		"City" : "Burbank",
		"State" : "CA"
	},
	"quantity" : 2
}
{
	"_id" : {
		"name" : "Sam",
		"ordered" : "turkey"
	},
	"address" : {
		"Street" : "Magnolia Blvd",
		"City" : "Burbank",
		"State" : "CA"
	},
	"quantity" : 3
}
{
	"_id" : {
		"name" : "Sam",
		"ordered" : "duck"
	},
	"address" : {
		"Street" : "Magnolia Blvd",
		"City" : "Burbank",
		"State" : "CA"
	},
	"quantity" : 2
}
{
	"_id" : {
		"name" : "Sam",
		"ordered" : "chicken"
	},
	"address" : {
		"Street" : "M

### Lets see which products are bought in greater quantities! AKA "Who loves joins?!"

#### Grouping prior to joining and creating an array of order: quantity key-value pairs.

In [18]:
mongo "db.orders.aggregate([{$group: { _id: '$_id.name', ordQuant: {$push: {order: '$_id.ordered', quantity: '$quantity'}} }}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : "Alejandra", "ordQuant" : [ { "order" : "shrimp", "quantity" : 2 }, { "order" : "beef", "quantity" : 2 } ] }
{ "_id" : "Sam", "ordQuant" : [ { "order" : "turkey", "quantity" : 3 }, { "order" : "duck", "quantity" : 2 }, { "order" : "chicken", "quantity" : 3 } ] }
{ "_id" : "Jess", "ordQuant" : [ { "order" : "beef", "quantity" : 2 }, { "order" : "lamb", "quantity" : 3 }, { "order" : "chicken", "quantity" : 4 } ] }
{ "_id" : "Bob", "ordQuant" : [ { "order" : "lamb", "quantity" : 2 }, { "order" : "veal", "quantity" : 2 }, { "order" : "chicken", "quantity" : 2 } ] }


#### Join and create new collection 

In [19]:
mongo "db.orders.aggregate([{$group: { _id: '$_id.name', ordQuant: {$push: {order: '$_id.ordered', quantity: '$quantity'}} }}, {$lookup: {from: 'customers', localField: '_id', foreignField: 'name', as: 'cust'}}]).pretty()"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{
	"_id" : "Alejandra",
	"ordQuant" : [
		{
			"order" : "shrimp",
			"quantity" : 2
		},
		{
			"order" : "beef",
			"quantity" : 2
		}
	],
	"cust" : [
		{
			"_id" : ObjectId("58cad8fd7832184696640ac7"),
			"name" : "Alejandra",
			"address" : {
				"Street" : "Segway Dr",
				"City" : "Bloomington",
				"State" : "IN"
			},
			"ordered" : [
				"shrimp",
				"beef"
			],
			"lifetime_value" : 2200
		}
	]
}
{
	"_id" : "Sam",
	"ordQuant" : [
		{
			"order" : "turkey",
			"quantity" : 3
		},
		{
			"order" : "duck",
			"quantity" : 2
		},
		{
			"order" : "chicken",
			"quantity" : 3
		}
	],
	"cust" : [
		{
			"_id" : ObjectId("58cad8fd7832184696640ac5"),
			"name" : "Sam",
			"address" : {
				"Street" : "Magnolia Blvd",
				"City" : "Burbank",
				"State" : "CA"
			},
			"ordered" : [
				"turkey",
				"duck",
				"chicken"
			],
			"lifetime_value" : 1000
		}
	]
}
{
	"_id" : "Jes

In [20]:
mongo "db.orders.aggregate([{$group: { _id: '$_id.name', ordQuant: {$push: {order: '$_id.ordered', quantity: '$quantity'}} }}, {$lookup: {from: 'customers', localField: '_id', foreignField: 'name', as: 'cust'}}, {$out: 'cmb'}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2


*Mongoose*
```javascript
app.get('/out1', function(req, res){
	mongoose.connection.db.collection('orders', function(err, collection){
		collection.aggregate([
				{$group: 
					{_id: '$_id.name', 
					ordQuant: {
						$push: {order: '$_id.ordered', quantity: '$quantity'}
					}}},
				{$lookup: {
					from: 'users',
					localField: '_id',
					foreignField: 'name',
					as: 'cust'
				}},
				{$out: 'cmb'}
			]).toArray(function(err, orders){
			if(err){
				console.log(err);
			} else {
				mongoose.connection.db.collection('orders', function(err, collection){
					collection.find({}).toArray(function(err, orders){
						if(err){
							console.log(err);
						} else {
							console.log(orders);
							return res.json(orders);
						}
					});
				});
			}
		});
	});
});
```

#### Unwind the new collection

In [21]:
mongo "db.cmb.aggregate([{$unwind: '$ordQuant'}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : "Alejandra", "ordQuant" : { "order" : "shrimp", "quantity" : 2 }, "cust" : [ { "_id" : ObjectId("58cad8fd7832184696640ac7"), "name" : "Alejandra", "address" : { "Street" : "Segway Dr", "City" : "Bloomington", "State" : "IN" }, "ordered" : [ "shrimp", "beef" ], "lifetime_value" : 2200 } ] }
{ "_id" : "Alejandra", "ordQuant" : { "order" : "beef", "quantity" : 2 }, "cust" : [ { "_id" : ObjectId("58cad8fd7832184696640ac7"), "name" : "Alejandra", "address" : { "Street" : "Segway Dr", "City" : "Bloomington", "State" : "IN" }, "ordered" : [ "shrimp", "beef" ], "lifetime_value" : 2200 } ] }
{ "_id" : "Sam", "ordQuant" : { "order" : "turkey", "quantity" : 3 }, "cust" : [ { "_id" : ObjectId("58cad8fd7832184696640ac5"), "name" : "Sam", "address" : { "Street" : "Magnolia Blvd", "City" : "Burbank", "State" : "CA" }, "ordered" : [ "turkey", "duck", "chicken" ], "lifetime_value" : 1000 }

#### Groups the products and averages out their quantities 

In [22]:
mongo "db.cmb.aggregate([{$unwind: '$ordQuant'}, {$group: {_id: '$ordQuant.order', average_quantity: {$avg: '$ordQuant.quantity'}}}])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : "lamb", "average_quantity" : 2.5 }
{ "_id" : "veal", "average_quantity" : 2 }
{ "_id" : "chicken", "average_quantity" : 3 }
{ "_id" : "turkey", "average_quantity" : 3 }
{ "_id" : "beef", "average_quantity" : 2 }
{ "_id" : "shrimp", "average_quantity" : 2 }
{ "_id" : "duck", "average_quantity" : 2 }


# AND FINALLY

#### SORT!(numerical and alphabetical)

In [23]:
mongo "db.cmb.aggregate([{$unwind: '$ordQuant'}, {$group: {_id: '$ordQuant.order', average_quantity: {$avg: '$ordQuant.quantity'}}}, {$sort: {average_quantity: -1, _id: 1}  }])"

MongoDB shell version v3.4.2
connecting to: mongodb://127.0.0.1:27017/Demo
MongoDB server version: 3.4.2
{ "_id" : "chicken", "average_quantity" : 3 }
{ "_id" : "turkey", "average_quantity" : 3 }
{ "_id" : "lamb", "average_quantity" : 2.5 }
{ "_id" : "beef", "average_quantity" : 2 }
{ "_id" : "duck", "average_quantity" : 2 }
{ "_id" : "shrimp", "average_quantity" : 2 }
{ "_id" : "veal", "average_quantity" : 2 }


*Mongoose*
```javascript
app.get('/group1', function(req, res){
	mongoose.connection.db.collection('cmb', function(err, collection){
		collection.aggregate([
				{$unwind: '$ordQuant'},
				{$group: {
					_id: '$ordQuant.order',
					average_quantity: {$avg: '$ordQuant.quantity'}
				}},
				{$sort: {average_quantity: -1, _id: 1}}
			]).toArray(function(err, orders){
			if(err){
				console.log(err);
			} else {
				console.log(orders);
				return res.json(orders);
			}
		});
	});	
});
```

In [ ]:
mongo "db.cmb.drop(); db.customers.drop(); db.orders.drop()"

In [ ]:
mongo "db.getCollectionNames()"